<a href="https://colab.research.google.com/github/johncclayton/jupyter-trading/blob/main/Triad_%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import time
import datetime
from dateutil import relativedelta


warnings.filterwarnings("ignore")



# Date range - start does not matter, end = tomorrow to make sure we get today's data in yahoo finance
start = '2018-01-01'
tom = datetime.date.today() + relativedelta.relativedelta(days=1)
end = tom.strftime("%Y-%m-%d")

# Tickers of assets
assets = ['IWB','IWS', 'VXUS','VCSH', 'VGSH', 'VGIT', 'SGOL', 'DBC']
assets.sort()

# Downloading data
data = yf.download(assets, start = start, end=end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets

[*********************100%%**********************]  8 of 8 completed


In [ ]:
pd.options.display.float_format = '{:.2f}'.format

score = data.resample('BM').last()
score['DBC_7m'] = score['DBC'].rolling(7).mean()
score['IWB_7m'] = score['IWB'].rolling(7).mean()
score['IWS_7m'] = score['IWS'].rolling(7).mean()
score['SGOL_7m'] = score['SGOL'].rolling(7).mean()
score['VCSH_7m'] = score['VCSH'].rolling(7).mean()
score['VGIT_7m'] = score['VGIT'].rolling(7).mean()
score['VGSH_7m'] = score['VGSH'].rolling(7).mean()
score['VXUS_7m'] = score['VXUS'].rolling(7).mean()


score

,DBC,IWB,IWS,SGOL,VCSH,VGIT,VGSH,VXUS,DBC_7m,IWB_7m,IWS_7m,SGOL_7m,VCSH_7m,VGIT_7m,VGSH_7m,VXUS_7m
Date,,,,,,,,,,,,,,,,
2018-01-31,15.74,141.57,80.31,12.97,66.66,54.38,52.56,49.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-28,15.28,136.33,76.31,12.75,66.30,54.13,52.53,46.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-30,15.63,133.28,76.50,12.83,66.39,54.51,52.63,46.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-30,16.16,133.62,76.85,12.77,66.32,54.03,52.54,47.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,16.60,137.06,77.74,12.58,66.56,54.45,52.74,46.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,23.34,275.29,118.48,21.89,75.55,56.49,56.84,58.43,22.69,263.08,113.78,20.04,75.07,57.06,56.69,56.38
2024-05-31,23.27,288.14,122.79,22.23,76.36,57.30,57.26,60.78,22.68,271.78,117.33,20.51,75.62,57.40,56.91,57.87
2024-06-28,23.23,297.54,120.74,22.22,76.76,57.87,57.58,60.30,22.74,278.80,119.25,20.90,75.97,57.58,57.10,58.70


In [ ]:
score['S1'] = ''
score['S2'] = ''
score['S3'] = ''
score['S4'] = ''

for i in assets:
  j = i + "_dist"
  h = i + "_7m"
  score[j] = (score[i] - score[h]) / score[i]




for i in range(8,len(score)):



  if ((score.iloc[i]['VCSH_dist'] > score.iloc[i]['VGSH_dist']) & (score.iloc[i]['VCSH_dist'] > score.iloc[i]['VGIT_dist'])):
    score.iloc[i,19] = 'VCSH'
  else:
    if ((score.iloc[i]['VGSH_dist'] > score.iloc[i]['VCSH_dist']) & (score.iloc[i]['VGSH_dist'] > score.iloc[i]['VGIT_dist'])):
      score.iloc[i,19] = 'VGSH'
    else:
      score.iloc[i,19] = 'VGIT'

  if score.iloc[i]['IWB_dist'] >= 0:
    score.iloc[i,16] = 'IWB'
  else:
    score.iloc[i,16] = score.iloc[i,19]

  if ((score.iloc[i]['IWS_dist'] >= 0 ) & (score.iloc[i]['IWS_dist'] >= score.iloc[i]['VXUS_dist'])):
    score.iloc[i,17] = 'IWS'
  else:
    if ((score.iloc[i]['VXUS_dist'] >= 0) & (score.iloc[i]['VXUS_dist'] >= score.iloc[i]['IWS_dist'])):
      score.iloc[i,17] = 'VXUS'
    else:
      score.iloc[i,17] = score.iloc[i,19]

  if ((score.iloc[i]['SGOL_dist'] >= 0) & (score.iloc[i]['SGOL_dist'] >= score.iloc[i]['DBC_dist'])):
    score.iloc[i,18] = 'SGOL'
  else:
    if ((score.iloc[i]['DBC_dist'] >= 0) & (score.iloc[i]['DBC_dist'] >= score.iloc[i]['SGOL_dist'])):
      score.iloc[i,18] = 'DBC'
    else:
      score.iloc[i,18] = score.iloc[i,19]

score

,DBC,IWB,IWS,SGOL,VCSH,VGIT,VGSH,VXUS,DBC_7m,IWB_7m,...,S3,S4,DBC_dist,IWB_dist,IWS_dist,SGOL_dist,VCSH_dist,VGIT_dist,VGSH_dist,VXUS_dist
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,15.74,141.57,80.31,12.97,66.66,54.38,52.56,49.57,NaN,NaN,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-28,15.28,136.33,76.31,12.75,66.30,54.13,52.53,46.97,NaN,NaN,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-30,15.63,133.28,76.50,12.83,66.39,54.51,52.63,46.80,NaN,NaN,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-30,16.16,133.62,76.85,12.77,66.32,54.03,52.54,47.01,NaN,NaN,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,16.60,137.06,77.74,12.58,66.56,54.45,52.74,46.24,NaN,NaN,...,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,23.34,275.29,118.48,21.89,75.55,56.49,56.84,58.43,22.69,263.08,...,SGOL,VCSH,0.03,0.04,0.04,0.08,0.01,-0.01,0.00,0.04
2024-05-31,23.27,288.14,122.79,22.23,76.36,57.30,57.26,60.78,22.68,271.78,...,SGOL,VCSH,0.03,0.06,0.04,0.08,0.01,-0.00,0.01,0.05
2024-06-28,23.23,297.54,120.74,22.22,76.76,57.87,57.58,60.30,22.74,278.80,...,SGOL,VCSH,0.02,0.06,0.01,0.06,0.01,0.01,0.01,0.03


In [ ]:
score['SmartLeverage'] = ''
pd.options.display.float_format = '{:.2f}'.format
monthlyhigh = data.resample('BM').last()
for i in range(1, len(monthlyhigh)):
  column = monthlyhigh["IWB"].head(i)
  max_value = column.max()
  max_index = column.idxmax()
  #print(max_value)
  #print(max_index)
  column2 = column[max_index:]
  #print(column2)
  min_value=column2.min()
  #print(min_value)
  dd = (max_value - min_value) / max_value
  #print(dd)
  #print(i)
  if dd>0.15:
    score.iloc[i,28] = True
  else:
    score.iloc[i,28] = False
score

,DBC,IWB,IWS,SGOL,VCSH,VGIT,VGSH,VXUS,DBC_7m,IWB_7m,...,S4,DBC_dist,IWB_dist,IWS_dist,SGOL_dist,VCSH_dist,VGIT_dist,VGSH_dist,VXUS_dist,SmartLeverage
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,15.74,141.57,80.31,12.97,66.66,54.38,52.56,49.57,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2018-02-28,15.28,136.33,76.31,12.75,66.30,54.13,52.53,46.97,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2018-03-30,15.63,133.28,76.50,12.83,66.39,54.51,52.63,46.80,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2018-04-30,16.16,133.62,76.85,12.77,66.32,54.03,52.54,47.01,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2018-05-31,16.60,137.06,77.74,12.58,66.56,54.45,52.74,46.24,NaN,NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,23.34,275.29,118.48,21.89,75.55,56.49,56.84,58.43,22.69,263.08,...,VCSH,0.03,0.04,0.04,0.08,0.01,-0.01,0.00,0.04,False
2024-05-31,23.27,288.14,122.79,22.23,76.36,57.30,57.26,60.78,22.68,271.78,...,VCSH,0.03,0.06,0.04,0.08,0.01,-0.00,0.01,0.05,False
2024-06-28,23.23,297.54,120.74,22.22,76.76,57.87,57.58,60.30,22.74,278.80,...,VCSH,0.02,0.06,0.01,0.06,0.01,0.01,0.01,0.03,False


In [ ]:
for i in range(8,len(score)):
  if score.iloc[i]['IWB_dist'] >= 0:
    if score.iloc[i-1]['S1'] == 'UPRO':
      score.iloc[i,16] = 'UPRO'
      WeContinue = False
      for j in range(13):
        if score.iloc[i-j,16] != 'UPRO':
          WeContinue = True
      if WeContinue == False:
        score.iloc[i,16] = 'IWB'
    else:
      #print(score.iloc[i])
      if score.iloc[i]['SmartLeverage'] == True:
        #print("True")
        score.iloc[i,16] = 'UPRO'
      else:
        score.iloc[i,16] = 'IWB'

In [ ]:
scorefinal = score.iloc[-36:,16:20]
scorefinal

,S1,S2,S3,S4
Date,,,,
2021-09-30,IWB,VCSH,DBC,VCSH
2021-10-29,IWB,IWS,DBC,VGSH
2021-11-30,IWB,VGSH,VGSH,VGSH
2021-12-31,IWB,IWS,DBC,VGSH
2022-01-31,VGSH,IWS,DBC,VGSH
2022-02-28,VGSH,VGSH,DBC,VGSH
2022-03-31,VGSH,IWS,DBC,VGSH
2022-04-29,VGSH,VGSH,DBC,VGSH
2022-05-31,VGSH,VGSH,DBC,VGSH


In [ ]:
scorefinal['Month']=scorefinal.index.shift(+1).strftime("%B %Y")
score2 = scorefinal
score2.reset_index(drop = True, inplace = True)
score3 = score2.sort_index(axis=1)
print("The investment selection for next month isn’t final until the close of the current month.")
print("Invest in 1/3 * S1, 1/3 * S2, 1/6 * S3 and 1/6 * S4\n\n")
score3[-12:]

The investment selection for next month isn’t final until the close of the current month.
Invest in 1/3 * S1, 1/3 * S2, 1/6 * S3 and 1/6 * S4




,Month,S1,S2,S3,S4
24,October 2023,VGSH,VGSH,DBC,VGSH
25,November 2023,VGSH,VGSH,DBC,VGSH
26,December 2023,UPRO,VXUS,SGOL,VCSH
27,January 2024,UPRO,IWS,SGOL,VGIT
28,February 2024,UPRO,IWS,SGOL,VGIT
29,March 2024,UPRO,IWS,SGOL,VCSH
30,April 2024,UPRO,IWS,SGOL,VCSH
31,May 2024,UPRO,IWS,SGOL,VCSH
32,June 2024,UPRO,VXUS,SGOL,VCSH
33,July 2024,UPRO,VXUS,SGOL,VCSH
